# Calculating population density

We want to know how many individuals are exposed to flares at each of the study regions.

To do this we want to:
- read in all population data from the study regions
- calculate the total population density within each of the study regions
-  using literatiure select an appropriate threshhold for AOD to use as a cutoff point for the "high levels of pollution"
- from this calculate total no. of people and % of people in area affected by high levels of pollution

In [24]:
import os

# Reading in all population data
import geopandas as gpd
base_dir = os.path.dirname(os.getcwd())

In [25]:
gwer_road = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_gwer_polygon.geojson")
kalak = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_kalak_polygon.geojson")
lalish = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_lalish_polygon.geojson")

In [26]:
gwer_road.head()

,VALUE,geometry
0,441.916229,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
1,419.327271,"MULTIPOLYGON (((43.78547 36.24193, 43.78977 36..."
2,511.886505,"MULTIPOLYGON (((43.74875 36.23515, 43.74875 36..."
3,441.208771,"MULTIPOLYGON (((43.75148 36.23655, 43.75708 36..."
4,245.390091,"MULTIPOLYGON (((43.76542 36.23875, 43.76542 36..."


In [39]:
sum(gwer_road["VALUE"])

121183.42779943

In [28]:
sum(gwer_road["VALUE"])

121183.42779943

In [29]:
sum(kalak["VALUE"])

219058.2630378402

In [30]:
sum(lalish["VALUE"])

40925.76306725002

Next we want to read in the `AOD` data which quanitifies atmospheric haziness and gives us an idea of the level of pollution in the atmosphere.


In [31]:
gwer_road_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_gwer_polygon.geojson")
kalak_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_kalak_polygon.geojson")
lalish_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_lalish_polygon.geojson")

In order to understand the exposure to `AOD` we need to select the each time the `AOD` was above a specific threshhold each month.

In [35]:
gwer_road_harmful_aod = gwer_road_aod[gwer_road_aod["Optical_Depth_047"]>175]
kalak_harmful_aod = kalak_aod[kalak_aod["Optical_Depth_047"]>175]
lalish_harmful_aod = lalish_aod[lalish_aod["Optical_Depth_047"]>175]

In [36]:
sum(gwer_road_harmful_aod["VALUE"])

2947635.1810786836

In [ ]:
kalak_harmful_aod